In [ ]:
import pandas as pd
import requests
import boto3
import json

pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', -1)

/var/folders/cq/4f424mlx5f1fdqtf5dwzmz8m0000gn/T/ipykernel_82100/2836181286.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
def last_cred():

    secret_name = "last_keys"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    last_key = json.loads(get_secret_value_response['SecretString'])['last_apiKEY']
    last_agent = json.loads(get_secret_value_response['SecretString'])['last_userAGENT']
    last_user = json.loads(get_secret_value_response['SecretString'])['last_username']

    return last_key, last_agent, last_user

In [ ]:
API_KEY, USER_AGENT, USERNAME = last_cred()

In [ ]:
df = pd.read_csv('track_features.csv', index_col=0)

In [ ]:
def last_tag(artist, method, tag_no=0, agent=USER_AGENT, user=USERNAME, key=API_KEY):

    headers = {
    'user-agent': agent
    }

    payload = {'method': method, 
                'artist': artist,
            'user': user,
            'api_key': key,
            'format': 'json'
                }


    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    try:
        top = r.json()['toptags']['tag']
        tags = [i['name'].lower() for i in top[:5]]
    except IndexError:
        tags = None
    return tags

In [ ]:
df['tag_list'] = df['artist'].apply(last_tag, method='artist.gettoptags')

In [ ]:
non_zero = df['tag_list'][df['tag_list'].apply(lambda x: len(x)) != 0]
gen_val = non_zero.apply(lambda x: x[0]).value_counts()
gen_list = gen_val[gen_val >= 10].index.to_list()
gen_list.remove('seen live')

In [ ]:
# gen_list

In [ ]:
def other_class(x):
    try:
        count = 0
        try_tag = x[count]
        while try_tag not in gen_list:
            count += 1
            try_tag = x[count]
    except IndexError:
        try_tag = 'Other'
    return try_tag

In [ ]:
df['genre'] = df['tag_list'].apply(other_class)

In [ ]:
df['genre'].value_counts()

electronic              256
hip-hop                 214
house                   104
indie                   87 
Other                   81 
minimal                 36 
rap                     32 
deep house              30 
soul                    29 
rnb                     24 
indie pop               21 
beats                   21 
dubstep                 21 
folk                    19 
trap                    19 
techno                  19 
indie rock              18 
lo-fi                   17 
progressive house       16 
ambient                 14 
dream pop               13 
chillwave               12 
hip hop                 12 
tech house              12 
chillout                10 
instrumental hip-hop    10 
Name: genre, dtype: int64

In [ ]:
df = df.drop('tag_list', axis=1)

In [ ]:
# df[['name', 'artist', 'genre']]

In [ ]:
df.to_csv('features_wGenre.csv')